In [1]:
import yfinance as yf
import pandas as pd
from scipy.stats import pearsonr

# 1. Define the Top 20 Tech Universe (2026 leaders)
tickers = [
    'NVDA', 'AAPL', 'GOOGL', 'MSFT', 'AMZN', 'TSM', 'META', 'AVGO', 'TSLA', 'ASML',
    'ORCL', 'NFLX', 'AMD', 'CRM', 'ADBE', 'PLTR', 'CSCO', 'IBM', 'TXN', 'INTC'
]

results = []

print("Fetching data for 20 tickers... This may take a moment.")

for symbol in tickers:
    try:
        t = yf.Ticker(symbol)
        info = t.info
        
        # Get Revenue Growth (TTM vs Prior Year)
        growth = info.get('revenueGrowth', 0)
        
        # Calculate 1-Year Return
        # Fetching roughly 1 year of history
        hist = t.history(period="1y")
        if len(hist) > 0:
            start_price = hist['Close'].iloc[0]
            end_price = hist['Close'].iloc[-1]
            one_year_return = (end_price - start_price) / start_price
            
            results.append({
                'Ticker': symbol,
                'Revenue_Growth': growth,
                'Return_1Y': one_year_return
            })
    except Exception as e:
        print(f"Skipping {symbol}: {e}")

# 2. Convert to DataFrame
df = pd.DataFrame(results)

# 3. Calculate Correlation and P-Value
# We drop any NaNs to ensure the math is clean
df = df.dropna()
r_val, p_val = pearsonr(df['Revenue_Growth'], df['Return_1Y'])

# 4. Output Results
print("\n--- Market Regime Analysis Results ---")
print(df.to_string(index=False))
print("-" * 38)
print(f"Correlation (r): {r_val:.4f}")
print(f"P-Value:         {p_val:.4f}")

# 5. Interpretation for your Report
if p_val < 0.05:
    print("\nVERDICT: Statistically Significant. The Growth-Regime theory holds.")
else:
    print("\nVERDICT: Not Significant. The correlation may be due to random noise.")

Fetching data for 20 tickers... This may take a moment.

--- Market Regime Analysis Results ---
Ticker  Revenue_Growth  Return_1Y
  NVDA           0.625   0.428379
  AAPL           0.157   0.227333
 GOOGL           0.180   0.748810
  MSFT           0.167  -0.013791
  AMZN           0.136  -0.082173
   TSM           0.205   0.715265
  META           0.238  -0.071341
  AVGO           0.164   0.493878
  TSLA          -0.031   0.136856
  ASML           0.049   0.960303
  ORCL           0.142  -0.173064
  NFLX           0.176  -0.189293
   AMD           0.341   0.937895
   CRM           0.086  -0.408897
  ADBE           0.105  -0.380285
  PLTR           0.700   0.225981
  CSCO           0.075   0.395229
   IBM           0.122   0.215240
   TXN           0.104   0.269070
  INTC          -0.041   1.648691
--------------------------------------
Correlation (r): -0.0480
P-Value:         0.8406

VERDICT: Not Significant. The correlation may be due to random noise.
